ログイン

In [1]:
#ログインIDとパスワード
USER = '*****'
PASS = '*****'
#対象年月(2021年9月分：year:'2021', month:'9')
year = '2022'
month = '12'
yearmonth = year + month
#メンティーの人数
mentee_number = 13
#mentee_list0に、メンティー社員を指定(表の一番上の社員を1、その次の社員を2,3,...として、番号で指定)
mentee_list0 = [2,3,4,5,6,7,8,9,10,11,12,13,14]
mentee_list = [i - 1 for i in mentee_list0]

#メンティーが月報を未提出の場合、メンター・メンティー報告書抽出時にtdタグがずれてエラーが起きる。

In [2]:
#ブラウザ操作を自動化(値をコピー、クリック、値を入力)
!pip install selenium
#HTMLを読み取る
!pip install beautifulsoup4
#Excelを操作
!pip install openpyxl

In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
#経過時間、処理時間の管理
import time
#データ解析
import pandas as pd
#HTML読み取り
from bs4 import BeautifulSoup
import urllib.request as req
import requests
#Pandas
import pandas as pd
#Excel
import openpyxl

In [4]:
#GoogleChromeを起動
browser = webdriver.Chrome(ChromeDriverManager().install())
browser.implicitly_wait(3)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 107.0.5304
[WDM] - Get LATEST chromedriver version for 107.0.5304 google-chrome
[WDM] - There is no [win32] chromedriver for browser 107.0.5304 in cache
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/107.0.5304.62/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\KAITO\.wdm\drivers\chromedriver\win32\107.0.5304.62]


In [5]:
#ログインするサイトへアクセス
url_login = 'https://www.*****'
browser.get(url_login)
#指定のurlに遷移する前に下の処理に行かないための記述
time.sleep(3)
print('ログインページにアクセスしました')

ログインページにアクセスしました


In [6]:
#テキストボックス入力
#HTMLのIDを指定して要素を取得
element = browser.find_element_by_id('user-id')
#テキストボックスの要素を空白にして、ログイン情報を入力
element.clear()
element.send_keys(USER)
element = browser.find_element_by_id('pw-id')
element.clear()
element.send_keys(PASS)
print('フォームを送信')

フォームを送信


In [7]:
#「ログイン」をクリック
browser_from = browser.find_element_by_xpath('/html/body/form/table/tbody/tr[6]/td[2]/input')
time.sleep(3)
browser_from.click()
print('情報を入力してログインボタンを押しました')

情報を入力してログインボタンを押しました


In [8]:
#cookieを記憶
cookie = browser.get_cookies()
cookies = {
    cookie['name']: cookie['value']
    for cookie in cookie}

In [9]:
#対象年月の月報一覧のページのhtmlを全て読み込む
url2 = 'https://www.*****' + yearmonth
response2 = requests.get(
    url2,
    cookies=cookies)
parse_html2 = BeautifulSoup(response2.content, 'html.parser')

In [10]:
#aタグのみを取り出し
title_lists2 = parse_html2.find_all('a')
#全メンバーの月報ページのリンクアイコン「開く」のみを取り出し
list(filter(lambda x: '開く' in x,title_lists2))
#表示URLの不足分を補い、全メンバーの月報ページのURLリストをurl_list2として作成
url_list2 = []
for i in list(filter(lambda x: '開く' in x,title_lists2)):
    url_list2.append(i.attrs['href'])
url_list2 = list(map(lambda x: 'https://www.*****' + x, url_list2))

In [14]:
#url_list2から大量のhtmlを読み込んで整理
parse_html_list2 = []
for i in url_list2:
    response2 = requests.get(i, cookies=cookies)
    parse_html2 = BeautifulSoup(response2.content, 'html.parser')
    parse_html_list2.append(parse_html2)
parse_html_list2

[<?xml version="1.0" encoding="utf-8"?>
 <!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN"  "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
 <html dir="ltr" lang="ja" xml:lang="ja" xmlns="http://www.w3.org/1999/xhtml">
 <head>
 <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
 <meta content="text/css" http-equiv="Content-Style-Type"/>
 <meta content="text/javascript" http-equiv="Content-Script-Type"/>
 <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
 <title>月次報告書へのフィードバック｜エウレカ MY PAGE</title>
 <meta content="MY PAGE" name="description"/>
 <meta content="MY PAGE" name="keywords"/>
 <meta content="width=1100" name="viewport"/>
 <link href="/eureka/eurmyp.nsf/css/import.css" rel="stylesheet" type="text/css"/>
 <link href="/eureka/eurmyp.nsf/css/index.css" rel="stylesheet" type="text/css"/>
 <script src="/eureka/eurmyp.nsf/js/external.js" type="text/javascript"></script>
 <script src="/eureka/eurmyp.nsf/js/jquery-1.9.0.min.js" type="text/javascript"><

In [15]:
#<td>タグのみ取り出す。未提出者のページは項目が合わなくなるので項目の数の空欄になるように処理。
title_lists2 = []
for i in parse_html_list2:
    title_lists2.append(i.find_all('td')[0:10])
for i in range(len(title_lists2)):
    if not title_lists2[i]:
        title_lists2[i] = ['','','','','','','','','','']
title_lists2

[[<td> ＜報連相について＞<br/>
  評価A。配属されて1ヶ月目で、分からないことも多かったので、業務の開始、途中始、途中経過、終わりでの報連相を意識して行動した。<br/>
  <br/>
  ＜ルール遵守について＞<br/>
  評価B。ルールが実験室によって違うので、覚えるのが大変だったが守ることができたと思う。今後、もう少し慣ていってルールを守って業務を遂行したい。 </td>,
  <td> 評価A。教わった担当業務をきちんと習得し、１人で行えるようになった。 </td>,
  <td> 評価B。周囲の人から協力を求められた際は、行動した。今後、言われる前に自発的に協力できるようにしたい。 </td>,
  <td> 評価A。就業先のデスクや、実験室は整理整頓を心がけた達成できた。今後も続けていきたいと思う。 </td>,
  <td> ＜仕事の量について＞<br/>
  評価B　就業開始して1ヶ月であったが、一通りの仕事量をこなせるようになった。ただ、定時時間いっぱい使ってしまうので、もう少し余裕がもてるようにしたい。<br/>
  <br/>
  ＜仕事の質について＞<br/>
  評価A。培地張込みでは菌のコンタミネーションもなく、また分析業務では問題ない結果がでた。また、就業先の方からも、丁寧で精度が高いという言葉もいただいた。今後は仕<br/>
  事量を増やしても質を保っていきたいと思う。 </td>,
  <td> 新しい業務を覚えることでいっぱいになってしまった。今後は、より業務を効率化できるように取り組んでいきたい。 </td>,
  <td> ①危険が想定される箇所は、実験室からの出入り口と通路の出合頭<br/>
  <br/>
  ②潜在事例<br/>
  <br/>
  ③転倒、衝突など<br/>
  <br/>
  ④原因：意識の欠如<br/>
  <br/>
  ⑤予想される被害：転倒や衝突による怪我（打撲、捻挫）。また、手に持っているものが割れやすいもの、危険なものであれば飛沫の可能性があり、そのもの自体も使用できなく<br/>
  なる。<br/>
  <br/>
  ⑥ルールの再認識、定期的にヒヤリとなる可能性を考える時間を作る。<br/>
  <br/>
  ⑦作製した培地等を違うフロアに持って行く際、台車に乗

In [16]:
#改行の際に現れる<br/>を取り除く。
title_list2 = []
for k in range(len(title_lists2)):
    for i in title_lists2[k]:
        if  i != '':
            title_list2.append(i.text.replace('<br/>', ''))
        else:
            title_list2.append(i)
title_list2

[' ＜報連相について＞\n評価A。配属されて1ヶ月目で、分からないことも多かったので、業務の開始、途中始、途中経過、終わりでの報連相を意識して行動した。\n\n＜ルール遵守について＞\n評価B。ルールが実験室によって違うので、覚えるのが大変だったが守ることができたと思う。今後、もう少し慣ていってルールを守って業務を遂行したい。 ',
 ' 評価A。教わった担当業務をきちんと習得し、１人で行えるようになった。 ',
 ' 評価B。周囲の人から協力を求められた際は、行動した。今後、言われる前に自発的に協力できるようにしたい。 ',
 ' 評価A。就業先のデスクや、実験室は整理整頓を心がけた達成できた。今後も続けていきたいと思う。 ',
 ' ＜仕事の量について＞\n評価B\u3000就業開始して1ヶ月であったが、一通りの仕事量をこなせるようになった。ただ、定時時間いっぱい使ってしまうので、もう少し余裕がもてるようにしたい。\n\n＜仕事の質について＞\n評価A。培地張込みでは菌のコンタミネーションもなく、また分析業務では問題ない結果がでた。また、就業先の方からも、丁寧で精度が高いという言葉もいただいた。今後は仕\n事量を増やしても質を保っていきたいと思う。 ',
 ' 新しい業務を覚えることでいっぱいになってしまった。今後は、より業務を効率化できるように取り組んでいきたい。 ',
 ' ①危険が想定される箇所は、実験室からの出入り口と通路の出合頭\n\n②潜在事例\n\n③転倒、衝突など\n\n④原因：意識の欠如\n\n⑤予想される被害：転倒や衝突による怪我（打撲、捻挫）。また、手に持っているものが割れやすいもの、危険なものであれば飛沫の可能性があり、そのもの自体も使用できなく\nなる。\n\n⑥ルールの再認識、定期的にヒヤリとなる可能性を考える時間を作る。\n\n⑦作製した培地等を違うフロアに持って行く際、台車に乗せエレベーターを使用して安全に運搬することができた。 ',
 ' 特になし ',
 ' 特になし ',
 ' 特になし ',
 ' 報連相について：評価A。口頭でお伝えする時には、何をどのような順番で言うかあらかじめ考え、はっきり話した。質問の答えに時間を要する場合にはすぐに伝え、十分な情報\nを揃えて返答やメールをした。その結果、わかりやすいと言っていた

In [17]:
#indexを入手
index_html = parse_html_list2[0].find_all('th')[0:10]
index_html2 = []
for i in index_html:
    index_html2.append(i.text.replace('<br/>', ''))
index_html2

['【規律性（報連相、ルール遵守）】就業先評価を確認し、自身の取組みを振返り、日々改善すべきこと、維持すべきことを考え、来月の行動目標を設定してください。（例）報連相について：指揮命令者へメール日報の報告をする際に、分かりやすくグラフをつけ、一目で結果の良し悪しが分かるように工夫した。ただ評価はBで変わらない為、「トラブルがあった際は細かいことでも報告してほしい」という定期訪問のコメントから、現場で異変に気付いた時は、細かなことも日報で報告していき、対面で報告できる時に口頭でも必ず報告するようにする。ルール遵守について：評価A。変わらずルール遵守を徹底していく。',
 '【責任性】就業先評価を確認し、自身の取組みを振返り、日々改善すべきこと、維持すべきことを考え、来月の行動目標を設定してください。（例）評価A。納期が設定されている場合は、納期通りに終わるスケジュールを計画・共有して、安心して頂くと共に必ず遵守する。スケジュール通りいかない場合は、分かった時点で報告をし、軌道修正を相談する。',
 '【協調性】就業先評価を確認し、自身の取組みを振返り、日々改善すべきこと、維持すべきことを考え、来月の行動目標を設定してください。（例）評価A。変わらず、誰でもできる仕事（試薬発注、掃除）は率先して行う。',
 '【マナー】就業先評価を確認し、自身の取組みを振返り、日々改善すべきこと、維持すべきことを考え、来月の行動目標を設定してください。（例）評価A。引き続き、誰よりも気持ちの良いオアシスを徹底する。',
 '【仕事の量・質】就業先評価を確認し、自身の取組みを振返り、日々改善すべきこと、維持すべきことを考え、来月の行動目標を設定してください。（例）量について：先月は、納期内にサンプル調整できるようスケジュール管理した。評価はBで変わらない為、先輩Aさんの処理量を参考にスケジュールの見直しを行う。納期より1日早い完了ができるよう挑戦する。質について：評価A。引き続き慣れた実験も作業手順を必ず確認し、イメージしてから実行することでミスを減らすよう心掛ける。量を意識した際にも怠らずに確認していく。',
 '【積極性】就業先評価を確認し、自身の取組みを振返り、日々改善すべきこと、維持すべきことを考え、来月の行動目標を設定してください。（例）評価A。前月からスケジュールと合わせて業

In [18]:
#Dataframe作成
#(columns:名前、index:項目説明、index.name)
DataFrame2 = pd.DataFrame(index=index_html2)
for i in range(int(len(title_lists2))):
    DataFrame2[parse_html_list2[i].find_all('p')[4].text[6:].replace('　', '')] = title_list2[10*i:10*(i+1)]
DataFrame2.index.name = '【対象者】'

In [20]:
#Excelファイルの月報担当分けから担当、入社年度、名前の組み合わせを入手
wb = openpyxl.load_workbook(filename = 'サビレポ担当分け(' + year + '.' + month +').xlsx')
df0 = pd.DataFrame(wb['Sheet1'].values)[1].dropna()[1:]
df0.name ='担当'
df1 = pd.DataFrame(wb['Sheet1'].values)[3].replace('・コース','').replace('ベーシック','').replace('ライフ','').replace('キャリア','').dropna()[2:]
df1.name = '入社年度'

In [21]:
#入社年度、担当の情報を持つDataFrameを新しく作成
df2 =pd.merge(df1, df0, right_index=True, left_index=True).transpose()
df2.columns=DataFrame2.columns.values
df2

,22E008：八尾夏子,22N017：新井千尋,22N022：菅原舜,22N034：谷川俊樹,22N057：佐藤こうみ,22N061：矢嶋恵美,22N062：神田梓,22N066：秋庭彩花,22N082：塩原由菜,22N096：大柳美裕,...,15N016：近恵,15N037：山口沙紀,15N167：堀川和男,14N011：大澤雪,14N072：上原理絵,13N050：滝沢涼子,13N079：髙以耒夏美,12N077：峠春菜,11E067：吉原健司,11N002：臼井明子
入社年度,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,...,2015,2015,2015,2014,2014,2013,2013,2012,2011,2011
担当,1,2,1,2,1,2,2,2,1,2,...,2,2,1,2,2,2,1,1,2,2


In [22]:
#月報のDataFrameに入社年度、担当情報のDataFrameを結合
DataFrame3 = pd.concat([DataFrame2, df2], axis=0) 
DataFrame3

,22E008：八尾夏子,22N017：新井千尋,22N022：菅原舜,22N034：谷川俊樹,22N057：佐藤こうみ,22N061：矢嶋恵美,22N062：神田梓,22N066：秋庭彩花,22N082：塩原由菜,22N096：大柳美裕,...,15N016：近恵,15N037：山口沙紀,15N167：堀川和男,14N011：大澤雪,14N072：上原理絵,13N050：滝沢涼子,13N079：髙以耒夏美,12N077：峠春菜,11E067：吉原健司,11N002：臼井明子
【規律性（報連相、ルール遵守）】就業先評価を確認し、自身の取組みを振返り、日々改善すべきこと、維持すべきことを考え、来月の行動目標を設定してください。（例）報連相について：指揮命令者へメール日報の報告をする際に、分かりやすくグラフをつけ、一目で結果の良し悪しが分かるように工夫した。ただ評価はBで変わらない為、「トラブルがあった際は細かいことでも報告してほしい」という定期訪問のコメントから、現場で異変に気付いた時は、細かなことも日報で報告していき、対面で報告できる時に口頭でも必ず報告するようにする。ルール遵守について：評価A。変わらずルール遵守を徹底していく。,＜報連相について＞\n評価A。配属されて1ヶ月目で、分からないことも多かったので、業務の開...,報連相について：評価A。口頭でお伝えする時には、何をどのような順番で言うかあらかじめ考え、...,評価A。\n忙しい時や予定より作業に時間がかかっているときに確認がおろそかにならないように...,規律性B\n報連相\n日々の実験業務に関しては報告まで十分にできているものの、資料作成など...,評価A\n報連相\n連絡内容と報告を求められる速度・重要度によって連絡ツール(電話・メール...,報連相について：指揮命令者へ報告する際、様子を伺い声をかけれなかったが、メールでの一報を入...,ルール遵守：評価B。定められているルールを遵守して業務を行った。しかし、少しでも変則的な場...,報連相について：急ぎではないものの、確認したいことが出た際、すぐにメモをとることで、怠りが...,報連相：評価はB。\n業務進捗に関する報連相は午前と午後には必ず行う。また、気になった点や...,報連相：評価A。ミスしてしまったことや完了の報告を適切なタイミングで行えたと思う。次月以降...,...,報連相について：評価A。お願いされていた測定が完了したら報告を行い、何か不明点があれば適宜...,報連相について：報告をする際に、分かりやすい文章を心がける\nルール遵守について：決まって...,報連相・ルール順守共にA評価\n適切にできている方だとは思うが、まだ何か質を上げることはで...,ルール順守について：評価はA。\n報連相について：評価はS。\n職場のルールはきちんと確認...,報連相：評価S。こまめな進捗報告を心がけた。タイミングは昼休み前後、終業前、トラブル時。結...,評価S\n常に職場にいらっしゃらない責任者の方にも日々のデータを見ることができるよう、デー...,,特に安全意識については保護具の着用をきちんと行い、怪我を未然に防ぐよう心掛けた。試薬の廃棄...,評価A\n 来月から就業先が変わるため、そこでのルールを身に着けることを徹底したいと思う。,報連相について：評価A。迅速な報告をこころがけた。報告の際は実験ごとにどの項目に注目したの...
【責任性】就業先評価を確認し、自身の取組みを振返り、日々改善すべきこと、維持すべきことを考え、来月の行動目標を設定してください。（例）評価A。納期が設定されている場合は、納期通りに終わるスケジュールを計画・共有して、安心して頂くと共に必ず遵守する。スケジュール通りいかない場合は、分かった時点で報告をし、軌道修正を相談する。,評価A。教わった担当業務をきちんと習得し、１人で行えるようになった。,評価A。仕事の指示を受けた場合は必ずその場で納期を確認し、計画的に実施した。また、突発的な...,評価A。\n一週間のスケジュールを紙に書きその日の終わりに次の日の予定を修正していく。,評価A\n優先順位の高いタスクを堅実にこなしているため、評価Aを頂けていると考えられる。一...,評価A\n社員さんが会議の多い日や出張中など、自分の作業の様子を社員さんが把握できない日は...,評価A。与えられた仕事のスケジュールを自分で計画し、共有することで周りの動きを把握出来安心...,評価A。自分の業務は最後まで責任をもって行うように意識した。複数人で受け持っている作業が多...,評価A。新しく行うことになった業務が、ほとんど一人で行う内容なので、教わったばかりである今...,責任性：評価はＢ。\n業務を行うにあたり、問題が発生した際は担当社員に即報告・相談し、軌道...,評価A。上司と実験スケジュールを立てる際に実験を進める順番の提案ができ、納期に間に合うよう...,...,評価A。任された業務に関しては遂行し完了報告までをセットとし、もし自分以外の人の協力が必要...,納期通りに終わるスケジュールを計画・共有して、安心して頂くと共に必ず遵守する,責任制：A評価\n自分の仕事は必ず納期よりも早く完了させている。また中途報告も適宜入れてお...,責任性について：評価はA。\n社員の方から与えられた業務はしっかり自分で最後まで完結できる...,評価S。新しい実験キットの依頼は上司の手順書を確認するのみだと結果を見ても考察ができなかっ...,評価S\n研究スケジュールを把握し、言われる前に「○日までにこれをしておく必要があります」...,,日々のタスクは予定通り終わるよう努めており信頼は頂けている。業務を終わらせることだけでなく...,評価A\n 納期が設定されている業務を優先させつつ、その中でなるべく早い業務遂行に努めてきた。,評価B。納期が設定されている業務はあまりなかったが、なるべく早い実施、報告をこころがけた。...
【協調性】就業先評価を確認し、自身の取組みを振返り、日々改善すべきこと、維持すべきことを考え、来月の行動目標を設定してください。（例）評価A。変わらず、誰でもできる仕事（試薬発注、掃除）は率先して行う。,評価B。周囲の人から協力を求められた際は、行動した。今後、言われる前に自発的に協力できるよ...,評価A。引き続き、自分の仕事が早く終わった時、周りが忙しそうにしている時には、積極的に手伝...,評価A。\n可能な限り雑務をこなす。,評価B\n頼まれた仕事はほぼ確実に引き受けるが、自分のタスクを依頼することが苦手。自分で処...,評価B\n共通業務(ゴミ捨て・備品の補充・4S)を隙間時間で積極的に行う,誰でも出来る仕事を率先して行うことを前提に、それ以外でも気がついたら試薬発注担当者に残量を...,評価A。手が空いた時には、他の人になにかできることがないか聞くようにした。今後もコミュニケ...,評価A。変わらず、乾燥棚の片付けや洗瓶の補充などのような、自分ができることを積極的に見つけ...,協調性：評価はB。\n雑務や小さい仕事、大きい仕事、やったことのない仕事もやりますと引き受...,評価A。気づいたときに消耗品の補充を行い、手が空いたときには他の方のフォローにも入ることが...,...,評価A。チーム内で急に仕事が振られたりする際に快い返事を意識するようにしたい。また、業務が...,共通バッファーの調製、消耗品の発注を率先して行う,協調性：A評価\n今まではやや自分の業務優先で雑務にあまり時間を割けていなかった。現在は少...,協調性について：評価はA。\n必要なコミュニケーションをとり、周りに気を配って行動していき...,評価S。消耗品の在庫過剰で倉庫が荒れており、倉庫整理を実施。消耗品の発注時期が曖昧なものに...,評価S\n他の人が忙しそうな時は「何かお手伝いできることはありますか」と声をかけるよう心掛...,,評価A\n試薬や備品の在庫管理を計画的に行い、実験が滞ることの無いようにする。共有の備品は...,評価A\n 新しい職場のルールを覚えることで、研究室全体の仕事などを分け合えるようにしたい。,評価B。協調性が求められる業務が発生しておらず、変化なし。備品在庫の管理や掃除などの雑事は...
【マナー】就業先評価を確認し、自身の取組みを振返り、日々改善すべきこと、維持すべきことを考え、来月の行動目標を設定してください。（例）評価A。引き続き、誰よりも気持ちの良いオアシスを徹底する。,評価A。就業先のデスクや、実験室は整理整頓を心がけた達成できた。今後も続けていきたいと思う。,評価A。職場の方や業者の方に、必ず真っ先に挨拶をする。,評価B。\n机の上を常に整理整頓するように心がける。,評価A\n継続して他者の見本になれるよう努める。,評価A\n引き続きオアシスの徹底を心がける。\n作業に集中するあまり挨拶し損ねることがあっ...,評価A。引き続き、誰よりも早く出勤し、雑務や気持ちの良いオアシスを徹底する。,評価A。挨拶やお礼など、基本的なコミュニケーションを忘れずに行った。しかし、挨拶やお礼を「...,評価A。引き続き、相手からではなく、自分から挨拶をしていくことを徹底していく。,マナー:：評価はA。\n上司や先輩方に正しい言葉使いが出来ること、清潔感のある身だしなみで...,評価A。出勤退勤のときはもちろんのこと、廊下ですれ違った際にも挨拶を心がけた。継続していき...,...,評

In [23]:
#担当1・２の振り分け
DataFrame_1 =pd.DataFrame()
DataFrame_2 =pd.DataFrame()
for i in range(len(title_lists2)):
    if DataFrame3.iat[11,i] == 1:
        DataFrame_1 = pd.concat([DataFrame_1,DataFrame3.iloc[:,i]], axis=1)
    else:
        DataFrame_2 = pd.concat([DataFrame_2, DataFrame3.iloc[:,i]], axis=1)

In [24]:
#入社年度の振り分け
DataFrame_1_2022 =pd.DataFrame()
DataFrame_1_20212020 =pd.DataFrame()
DataFrame_1_2019 =pd.DataFrame()
for i in range(len(DataFrame_1.columns)):
    if DataFrame_1.iat[10,i] == 2022:
        DataFrame_1_2022 = pd.concat([DataFrame_1_2022,DataFrame_1.iloc[:,i]], axis=1)
    elif DataFrame_1.iat[10,i] == 2021 or DataFrame_1.iat[10,i] == 2020:
        DataFrame_1_20212020 = pd.concat([DataFrame_1_20212020, DataFrame_1.iloc[:,i]], axis=1)
    else:
        DataFrame_1_2019 = pd.concat([DataFrame_1_2019,DataFrame_1.iloc[:,i]], axis=1)   
        
DataFrame_2_2022 =pd.DataFrame()
DataFrame_2_20212020 =pd.DataFrame()
DataFrame_2_2019 =pd.DataFrame()
for i in range(len(DataFrame_2.columns)):
    if DataFrame_2.iat[10,i] == 2022:
        DataFrame_2_2022 = pd.concat([DataFrame_2_2022,DataFrame_2.iloc[:,i]], axis=1)
    elif DataFrame_2.iat[10,i] == 2021 or DataFrame_2.iat[10,i] == 2020:
        DataFrame_2_20212020 = pd.concat([DataFrame_2_20212020, DataFrame_2.iloc[:,i]], axis=1)
    else:
        DataFrame_2_2019 = pd.concat([DataFrame_2_2019,DataFrame_2.iloc[:,i]], axis=1) 

メンターメンティー報告書のページの作成

In [28]:
#メンターメンティー報告書を全て入手
parse_html_list3 = []
for i in mentee_list:
    #メンティーの報告書
    parse_html_list3.append(parse_html_list2[i].find_all('td')[12:28:2])
    #メンティーのページにあるメンターの報告書
    parse_html_list3.append(parse_html_list2[i].find_all('td')[13:28:2])
for i in range(len(parse_html_list3)):
    if parse_html_list3[i] == []:
        parse_html_list3[i] = ['','','','','','','','']

parse_html_list3_replace = []
for k in range(8):
    for i in parse_html_list3:
        if  i[k] != '':
            parse_html_list3_replace.append(i[k].text.replace('<td>', '').replace('</td>', '').replace('\u3000', '').replace('\n', ''))
        else:
            parse_html_list3_replace.append(i[k])
parse_html_list3_replace

['メンティ：新井千尋',
 'メンター：大澤 雪',
 'メンティ：菅原舜',
 'メンター：佐藤 佑樹',
 'メンティ：谷川俊樹',
 'メンター：吉原 健司',
 'メンティ：佐藤こうみ',
 'メンター：渡邊 杏弥',
 'メンティ：矢嶋恵美',
 'メンター：大澤 雪',
 'メンティ：神田梓',
 'メンター：上原 理絵',
 'メンティ：秋庭彩花',
 'メンター：峠 春菜',
 'メンティ：塩原由菜',
 'メンター：阿部 真由子',
 'メンティ：大柳美裕',
 'メンター：北舘 奏子',
 'メンティ：熊坂健太',
 'メンター：堀川 和男',
 'メンティ：立田玲己',
 'メンター：尾﨑 開斗',
 'メンティ：田口啓護',
 'メンター：堀川 和男',
 'メンティ：牧野祐奈',
 'メンター：上原 理絵',
 'メール',
 'メール',
 'メール',
 'メール',
 'メール',
 'メール',
 'メール',
 'メール',
 'メール',
 'メール',
 'メール',
 'メール',
 'メール',
 'メール',
 'メール',
 'メール',
 'メール',
 'メール',
 'メール',
 'メール',
 'メール',
 'メール',
 'メール',
 'メール',
 'メール',
 'メール',
 '十分に受けられた',
 'あまりできなかった',
 '十分に受けられた',
 'おおむねできた',
 '十分に受けられた',
 'おおむねできた',
 '十分に受けられた',
 '十分にできた',
 'おおむね受けられた',
 'おおむねできた',
 'おおむね受けられた',
 'おおむねできた',
 '十分に受けられた',
 'あまりできなかった',
 '十分に受けられた',
 'おおむねできた',
 'おおむね受けられた',
 'おおむねできた',
 '十分に受けられた',
 'おおむねできた',
 'おおむね受けられた',
 'おおむねできた',
 'おおむね受けられた',
 'おおむねできた',
 '十分に受けられた',
 '十分にできた',
 '良好',
 '良好',
 '良好',
 '問題なし',
 '良好',
 '問題なし',
 '良好',
 '良好',
 '良好',
 '良好',
 '問題なし',
 '良

In [29]:
#メンターメンティー報告書を全て入手
parse_html_list3 = []
for i in mentee_list:
    #メンティーの報告書
    parse_html_list3.append(parse_html_list2[i].find_all('td')[12:28:2])
    #メンティーのページにあるメンターの報告書
    parse_html_list3.append(parse_html_list2[i].find_all('td')[13:28:2])
for i in range(len(parse_html_list3)):
    if parse_html_list3[i] == []:
        parse_html_list3[i] = ['','','','','','','','']
parse_html_list3

[[<td>メンティ：新井　千尋</td>,
  <td>メール</td>,
  <td>十分に受けられた</td>,
  <td>良好</td>,
  <td>特にありません</td>,
  <td>満足している</td>,
  <td>現状維持</td>,
  <td></td>],
 [<td>メンター：大澤 雪</td>,
  <td>メール</td>,
  <td>あまりできなかった</td>,
  <td>良好</td>,
  <td>先月の月報や定例会後の行動目標のフィードバックでメールを送りました。特に不安なことは無いですが、社会人基礎コースなど社内活動への参加もあって忙しいかもしれないの
  でこちらから積極的にトークで連絡してあげたいとおもいます。</td>,
  <td>満足している</td>,
  <td>現状維持</td>,
  <td>特にありません。</td>],
 [<td>メンティ：菅原　舜</td>,
  <td>メール</td>,
  <td>十分に受けられた</td>,
  <td>良好</td>,
  <td>寒くなってきたので体調管理に気を付けていきたいと思います</td>,
  <td>満足している</td>,
  <td>現状維持</td>,
  <td></td>],
 [<td>メンター：佐藤 佑樹</td>,
  <td>メール</td>,
  <td>おおむねできた</td>,
  <td>問題なし</td>,
  <td>季節の変わり目なので体調を崩さないか心配しています。</td>,
  <td>普通</td>,
  <td>現状維持</td>,
  <td></td>],
 [<td>メンティ：谷川　俊樹</td>,
  <td>メール</td>,
  <td>十分に受けられた</td>,
  <td>良好</td>,
  <td>最近は業務量が多くなっています。タスク管理について試行錯誤中です。</td>,
  <td>満足している</td>,
  <td>現状維持</td>,
  <td></td>],
 [<td>メンター：吉原 健司</td>,
  <td>メール</td>,
  <td>おおむねできた</td>,
  <td>問題なし</td>,
  <td>報告資料作成等が詰まって忙しくしてい

In [30]:
#indexを入手
index_html3 = parse_html_list2[mentee_list[0]].find_all('th')[10:18]
index_html4 = []
for i in index_html3:
    index_html4.append(i.text.replace('<br/>', ''))
index_html4

['入力者',
 '連絡状況',
 'メンターのフォロー',
 'メンティ・メンター間の関係性',
 'メンターの近況報告',
 'メンター制度への満足度',
 'メンターメンティに対する要望',
 '相談記入欄']

In [31]:
#Dataframe作成(index:項目説明)
DataFrame4 = pd.DataFrame()
for i in range(8):
    DataFrame4[i]= parse_html_list3_replace[int(len(parse_html_list3_replace)/8*i):int(len(parse_html_list3_replace)/8*(i+1))]
DataFrame4 = DataFrame4.reset_index(drop=True).T
DataFrame4.index = index_html4
DataFrame4

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
入力者,メンティ：新井千尋,メンター：大澤 雪,メンティ：菅原舜,メンター：佐藤 佑樹,メンティ：谷川俊樹,メンター：吉原 健司,メンティ：佐藤こうみ,メンター：渡邊 杏弥,メンティ：矢嶋恵美,メンター：大澤 雪,...,メンティ：大柳美裕,メンター：北舘 奏子,メンティ：熊坂健太,メンター：堀川 和男,メンティ：立田玲己,メンター：尾﨑 開斗,メンティ：田口啓護,メンター：堀川 和男,メンティ：牧野祐奈,メンター：上原 理絵
連絡状況,メール,メール,メール,メール,メール,メール,メール,メール,メール,メール,...,メール,メール,メール,メール,メール,メール,メール,メール,メール,メール
メンターのフォロー,十分に受けられた,あまりできなかった,十分に受けられた,おおむねできた,十分に受けられた,おおむねできた,十分に受けられた,十分にできた,おおむね受けられた,おおむねできた,...,おおむね受けられた,おおむねできた,十分に受けられた,おおむねできた,おおむね受けられた,おおむねできた,おおむね受けられた,おおむねできた,十分に受けられた,十分にできた
メンティ・メンター間の関係性,良好,良好,良好,問題なし,良好,問題なし,良好,良好,良好,良好,...,問題なし,問題なし,良好,問題なし,問題なし,問題なし,問題なし,問題なし,良好,良好
メンターの近況報告,特にありません,先月の月報や定例会後の行動目標のフィードバックでメールを送りました。特に不安なことは無いです...,寒くなってきたので体調管理に気を付けていきたいと思います,季節の変わり目なので体調を崩さないか心配しています。,最近は業務量が多くなっています。タスク管理について試行錯誤中です。,報告資料作成等が詰まって忙しくしている様子,能力開発プログラムの社会人基礎コースが11月末で終了のため、ひと段落つきそうです。引き続きよ...,出来なかったことを具体的に書き出し、来月の行動目標がしっかりと記入できていると思います。評価...,最近は上司の論文締切期日が近いため、実験だけでなくデータのまとめを行っている。,こちらから積極的に連絡をとることができませんでしたが、メンティの方から社内の手続きに関して質...,...,隣の部署に同期のエウレカ社員が週に数回来るようになり、交流ができるようになりました。,ケアレスミスはあるようですが、めげずに自分なりに工夫・改善を目指して頑張っているようです。,最近は残業が少し減りました。,特別問題はなさそうな状況である。ただ、今月から変更となった月報の内容からは以前以上に状況を推...,最近は少し落ち着いて定時で上がれる日が増えてきています。,ちょくちょく会社でも見かけます。うまくやっています。,最近暖かくて助かりました。冬要りません。,これといった問題は見受けられない。彼に限った話ではないが、月報の内容があっさりしすぎており、...,特に変わりありません。,評価がなかなか上がらないことで悩んでいたようです。何故上がらないのか本人納得いかない部分があ...
メンター制度への満足度,満足している,満足している,満足している,普通,満足している,普通,満足している,満足している,満足している,満足している,...,満足している,普通,満足している,普通,普通,普通,普通,普通,満足している,満足している
メンターメンティに対する要望,現状維持,現状維持,現状維持,現状維持,現状維持,現状維持,現状維持,現状維持,現状維持,現状維持,...,現状維持,現状維持,現状維持,現状維持,現状維持,現状維持,現状維持,現状維持,現状維持,現状維持
相談記入欄,,特にありません。,,,,,,,,特にありません。,...,,特になし。,,特になし,,,,特になし,,今月で牧野さんは1年経つのですが、このメンター報告の月報欄は削除されるのでしょうか？何か手続...


In [32]:
#入社年度、担当情報df3をメンターメンティー報告書用に再構築
df3 = pd.merge(df1, df0, right_index=True, left_index=True).reset_index(drop=True).T
df4 = pd.DataFrame()
for i in mentee_list :
    df4 = pd.concat([df4,df3.iloc[:,i]], axis=1)
    df4 = pd.concat([df4,df3.iloc[:,i]], axis=1)
df4.columns=range(df4.shape[1])
df4

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
入社年度,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,...,2022,2022,2022,2022,2022,2022,2022,2022,2021,2021
担当,2,2,1,1,2,2,1,1,2,2,...,2,2,1,1,1,1,1,1,2,2


In [33]:
#メンターメンティー報告書のDataFrameに入社年度、担当情報を追加
DataFrame5 = pd.concat([DataFrame4, df4], axis=0) 
DataFrame5.index.name = 'メンター・メンティ報告書'
DataFrame5

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
メンター・メンティ報告書,,,,,,,,,,,,,,,,,,,,,
入力者,メンティ：新井千尋,メンター：大澤 雪,メンティ：菅原舜,メンター：佐藤 佑樹,メンティ：谷川俊樹,メンター：吉原 健司,メンティ：佐藤こうみ,メンター：渡邊 杏弥,メンティ：矢嶋恵美,メンター：大澤 雪,...,メンティ：大柳美裕,メンター：北舘 奏子,メンティ：熊坂健太,メンター：堀川 和男,メンティ：立田玲己,メンター：尾﨑 開斗,メンティ：田口啓護,メンター：堀川 和男,メンティ：牧野祐奈,メンター：上原 理絵
連絡状況,メール,メール,メール,メール,メール,メール,メール,メール,メール,メール,...,メール,メール,メール,メール,メール,メール,メール,メール,メール,メール
メンターのフォロー,十分に受けられた,あまりできなかった,十分に受けられた,おおむねできた,十分に受けられた,おおむねできた,十分に受けられた,十分にできた,おおむね受けられた,おおむねできた,...,おおむね受けられた,おおむねできた,十分に受けられた,おおむねできた,おおむね受けられた,おおむねできた,おおむね受けられた,おおむねできた,十分に受けられた,十分にできた
メンティ・メンター間の関係性,良好,良好,良好,問題なし,良好,問題なし,良好,良好,良好,良好,...,問題なし,問題なし,良好,問題なし,問題なし,問題なし,問題なし,問題なし,良好,良好
メンターの近況報告,特にありません,先月の月報や定例会後の行動目標のフィードバックでメールを送りました。特に不安なことは無いです...,寒くなってきたので体調管理に気を付けていきたいと思います,季節の変わり目なので体調を崩さないか心配しています。,最近は業務量が多くなっています。タスク管理について試行錯誤中です。,報告資料作成等が詰まって忙しくしている様子,能力開発プログラムの社会人基礎コースが11月末で終了のため、ひと段落つきそうです。引き続きよ...,出来なかったことを具体的に書き出し、来月の行動目標がしっかりと記入できていると思います。評価...,最近は上司の論文締切期日が近いため、実験だけでなくデータのまとめを行っている。,こちらから積極的に連絡をとることができませんでしたが、メンティの方から社内の手続きに関して質...,...,隣の部署に同期のエウレカ社員が週に数回来るようになり、交流ができるようになりました。,ケアレスミスはあるようですが、めげずに自分なりに工夫・改善を目指して頑張っているようです。,最近は残業が少し減りました。,特別問題はなさそうな状況である。ただ、今月から変更となった月報の内容からは以前以上に状況を推...,最近は少し落ち着いて定時で上がれる日が増えてきています。,ちょくちょく会社でも見かけます。うまくやっています。,最近暖かくて助かりました。冬要りません。,これといった問題は見受けられない。彼に限った話ではないが、月報の内容があっさりしすぎており、...,特に変わりありません。,評価がなかなか上がらないことで悩んでいたようです。何故上がらないのか本人納得いかない部分があ...
メンター制度への満足度,満足している,満足している,満足している,普通,満足している,普通,満足している,満足している,満足している,満足している,...,満足している,普通,満足している,普通,普通,普通,普通,普通,満足している,満足している
メンターメンティに対する要望,現状維持,現状維持,現状維持,現状維持,現状維持,現状維持,現状維持,現状維持,現状維持,現状維持,...,現状維持,現状維持,現状維持,現状維持,現状維持,現状維持,現状維持,現状維持,現状維持,現状維持
相談記入欄,,特にありません。,,,,,,,,特にありません。,...,,特になし。,,特になし,,,,特になし,,今月で牧野さんは1年経つのですが、このメンター報告の月報欄は削除されるのでしょうか？何か手続...
入社年度,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022,...,2022,2022,2022,2022,2022,2022,2022,2022,2021,2021


In [34]:
#担当1・２の振り分け
DataFrame5_1 =pd.DataFrame()
DataFrame5_2 =pd.DataFrame()
for i in range(mentee_number *2):
    if DataFrame5.iat[9,i] == 1:
        DataFrame5_1 = pd.concat([DataFrame5_1,DataFrame5.iloc[:,i]], axis=1)
    else:
        DataFrame5_2 = pd.concat([DataFrame5_2, DataFrame5.iloc[:,i]], axis=1)

In [35]:
#index.nameを決定
DataFrame_1_2022.index.name = '【対象者】'
DataFrame_1_20212020.index.name = '【対象者】'
DataFrame_1_2019.index.name = '【対象者】'
DataFrame_2_2022.index.name = '【対象者】'
DataFrame_2_20212020.index.name = '【対象者】'
DataFrame_2_2019.index.name = '【対象者】'

In [36]:
#所定のExcelにシート作成
with pd.ExcelWriter('【川崎1・2エリア】【●年度●月】エリア状況報告書.xlsx',engine='openpyxl', mode = 'a') as writer:
    DataFrame_1_2022.iloc[:10,:].to_excel(writer, sheet_name="22入社(1)")
    DataFrame_2_2022.iloc[:10,:].to_excel(writer, sheet_name="22入社(2)")
    DataFrame_1_20212020.iloc[:10,:].to_excel(writer, sheet_name="21-20入社(1)")
    DataFrame_2_20212020.iloc[:10,:].to_excel(writer, sheet_name="21-20入社(2)")
    DataFrame_1_2019.iloc[:10,:].to_excel(writer, sheet_name="19以前入社(1)")
    DataFrame_2_2019.iloc[:10,:].to_excel(writer, sheet_name="19以前入社(2)")
    DataFrame5_1.iloc[:8,:].to_excel(writer, sheet_name="メンターメンティー報告書(1)")
    DataFrame5_2.iloc[:8,:].to_excel(writer, sheet_name="メンターメンティー報告書(2)")

In [37]:
#Excel編集(列幅設定、中央揃え、折り返して表示、セル背景色設定)
from openpyxl import load_workbook
from openpyxl.styles import Alignment, PatternFill
import string
from openpyxl.workbook.protection import WorkbookProtection

wb = load_workbook('【川崎1・2エリア】【●年度●月】エリア状況報告書.xlsx')
for sheet in wb.worksheets[2:5]:
    for i in range(12):
        sheet.row_dimensions[i].height= 100
        for A in string.ascii_letters[26:51]:
            sheet.column_dimensions[A].width = 50
    for row in sheet['A1:Z11']:
        for cell in row:
            cell.alignment = Alignment(horizontal = 'center', vertical = 'center', wrap_text=True)
for i in range(10):
    wb.worksheets[5].row_dimensions[i].height= 30
for A in string.ascii_letters[26:51]:
    wb.worksheets[5].column_dimensions[A].width = 20
for row in wb.worksheets[5]['A1:Z11']:
    for cell in row:
        cell.alignment = Alignment(horizontal = 'center', vertical = 'center', wrap_text=True)
for row in wb.worksheets[5]['D2:E9']:
    for cell in row:
        cell.fill =  PatternFill(fgColor='C6E0B4', fill_type='solid')
for row in wb.worksheets[5]['H2:I9']:
    for cell in row:
        cell.fill =  PatternFill(fgColor='C6E0B4', fill_type='solid')
for row in wb.worksheets[5]['L2:M9']:
    for cell in row:
        cell.fill =  PatternFill(fgColor='C6E0B4', fill_type='solid')
wb.save('【川崎1・2エリア】【' + year +'年度' + month + '月】エリア状況報告書.xlsx')

In [38]:
#【川崎1・2エリア】【●年度●月】エリア状況報告書に追加されたシートを削除して元に戻す。
wb = load_workbook('【川崎1・2エリア】【●年度●月】エリア状況報告書.xlsx')
wb.remove_sheet(wb.get_sheet_by_name("22入社"))
wb.remove_sheet(wb.get_sheet_by_name("21-20入社"))
wb.remove_sheet(wb.get_sheet_by_name("19以前入社"))
wb.remove_sheet(wb.get_sheet_by_name("メンターメンティー報告書"))
wb.save('【川崎1・2エリア】【●年度●月】エリア状況報告書.xlsx')

<ipython-input-38-b347a9de6ca1>:4: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  wb.remove_sheet(wb.get_sheet_by_name("22入社"))
<ipython-input-38-b347a9de6ca1>:4: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  wb.remove_sheet(wb.get_sheet_by_name("22入社"))
<ipython-input-38-b347a9de6ca1>:5: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  wb.remove_sheet(wb.get_sheet_by_name("21-20入社"))
<ipython-input-38-b347a9de6ca1>:5: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  wb.remove_sheet(wb.get_sheet_by_name("21-20入社"))
<ipython-input-38-b347a9de6ca1>:6: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  wb.remove_sheet(wb.get_sheet_by_name("19以前入社"))
<ipython-input-38-b347a9de6ca1>:6: DeprecationWarning: Call to deprecated function remove_sheet 